## Demo RPA

In [2]:
import rpa as r

r.init()
r.url('https://www.google.com')
r.type('//*[@name="q"]', 'decentralisation[enter]')
print(r.read('result-stats'))
r.snap('page', 'results.png')
r.close()

[RPA][INFO] - setting up TagUI for use in your Python environment
[RPA][INFO] - downloading TagUI (~200MB) and unzipping to below folder...
[RPA][INFO] - /Users/lyqht
[RPA][INFO] - done. syncing TagUI with stable cutting edge version
[RPA][INFO] - downloading latest PhantomJS to fix OpenSSL issue
[RPA][INFO] - TagUI now ready for use in your Python environment
About 59,100,000 results (0.53 seconds)


True

## Scraping from AXS

An example of getting bill payee from a specific category. As you can see, the selectors are ugly and the DOM structure looks weird af.

In [5]:
import rpa as r
numCategories = 10

billSideBarEl = '#divBillsPanel>table>tbody>tr>td>#left_nav:nth-child(1)'

def getBillSidebarItem(index = 0):
    return billSideBarEl + '>ul>li:nth-child(%s)' % (index+1)

def getBillPayeeInRowIndex(index = 0):
    rowElTemplate = 'table:nth-child(2) > tbody > tr > td:nth-child(3) > table:nth-child(%s)'
    return rowElTemplate % ((index+1)*2) + '> tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1)'

r.init(visual_automation = True)
r.url('https://e-station2.axs.com.sg/')
r.click('Pay Bills')
r.click(billSideBarEl)


category = r.read(getBillSidebarItem(0))
r.click(getBillSidebarItem(0))
print("Getting billing organisations of category: %s" % category)

payee = r.read(getBillPayeeInRowIndex(1))
print("Second payee in category: %s" % payee)

r.close()

Getting billing organisations of category: General
Second payee in category Singtel


True

## Scraping from POSB

Working example of getting all bill payees from available categories listed on the website.

In [11]:
import rpa as r
import requests
import pandas as pd
import numpy as np

url = 'https://www.posb.com.sg/personal/deposits/pay-with-ease/local-bill-payments/giro-payee-organisations.page'
html = requests.get(url).content

df_list = pd.read_html(html)
num_categories = len(df_list)
print("Found %s categories" % len(df_list))

Found 9 categories


In [16]:
df_list[0]

df_list[0].columns.values

array(['Billing Organisation', 'Bill Reference Number', 'Field Length',
       'Example'], dtype=object)

### Getting Payees without category

In [19]:
combined_array = np.concatenate(df_list)
df = pd.DataFrame(combined_array)
df.to_csv('payees_without_category.csv', 
          header=list(df_list[0].columns.values),
          index=False)

df

,0,1,2,3
0,DBS Vickers Securities (Singapore) Pte Ltd,DBS Vickers Trading account number,7 Digits,"80XX XXX Please indicate ""0"" for payment limit."
1,67 Pall Mall Singapore Pte. Ltd.,Membership No.,9 Characters (all numeric),012345678
2,SENTOSA GOLF CLUB,Membership Number,7 Alphanumeric Characters (Maximum),A12345 / A12345X / AB1234
3,Singapore Swimming Club,Account No. with billing corporation as shown ...,5 Characters (2 Alphabet 3 Digits),TA123
4,Singapore Island Country Club,Club No. as provided by billing organisation,5 Characters (1 Alphabet 4 Digits),A0123400 as A1234
...,...,...,...,...
99,StarHub Ltd,Account No. with StarHub Ltd as displayed on y...,"Numeric with decimals, ending with a letter.",Eg: 1.12345678A
100,Sembcorp Power Pte Ltd,Account No. with Sembcorp Power Pte Ltd as dis...,10 Characters (1 Uppercase alphabet & 9 digits),H123456782 / B000000018
101,Sunseap Energy Pte Ltd,Sunseap Account number as shown on top right o...,Sunseap Account number as shown on top right o...,23012345
102,Union Power Pte Ltd,MSSL Number,10 Digits / 13 Digits,9308014458 9308014458-013


### Getting categories with RPA

In [3]:
r.init()
r.url(url)
categories = [r.read(".rich-text-box>div:nth-child(%s)>h2" % (3+x*2)) for x in range(num_categories)]

print(categories)

r.close()

['Brokerage / Securities Firms', 'Country Clubs / Recreational Clubs', 'Credit Cards / DBS Cashline', 'Educational Institution', 'Government Agencies', 'Insurance Companies', 'Other Agencies', 'Telecommunications and Utilities', 'Town Councils']


True

### Example of Adding Category to just 1 table

In [4]:
df = df_list[2]
df['Category'] = categories[2]
df

,Billing Organisation,Bill Reference Number,Field Length,Example,Category
0,American Express Card,"For credit cards (3762 1 xxx...), input first ...",11 Digits,3762 123456 78901 input the bill reference no....,Credit Cards / DBS Cashline
1,American Express Card,"For charge cards (3762 0 xxx...), input first ...",11 Digits,3762 123456 78901 input the bill reference no....,Credit Cards / DBS Cashline
2,American Express Card,"For corporate cards (3762 2 xxx...), input fir...",11 Digits,3762 123456 78901 input the bill reference no....,Credit Cards / DBS Cashline
3,DBS Cashline (information),10-Digit DBS Cashline account no.,10 Digits,0821234567,Credit Cards / DBS Cashline


### Add Category to all payees

In [20]:
for x in range(num_categories):
    df_list[x]['Category'] = categories[x]

combined_array = np.concatenate(df_list)

df = pd.DataFrame(combined_array)
df.to_csv('payees_with_category.csv',
          header=list(df_list[0].columns.values),
          index=False)

df.to_excel('payees_with_category.xlsx',
            header=list(df_list[0].columns.values),
            index=False)

df

,0,1,2,3,4
0,DBS Vickers Securities (Singapore) Pte Ltd,DBS Vickers Trading account number,7 Digits,"80XX XXX Please indicate ""0"" for payment limit.",Brokerage / Securities Firms
1,67 Pall Mall Singapore Pte. Ltd.,Membership No.,9 Characters (all numeric),012345678,Country Clubs / Recreational Clubs
2,SENTOSA GOLF CLUB,Membership Number,7 Alphanumeric Characters (Maximum),A12345 / A12345X / AB1234,Country Clubs / Recreational Clubs
3,Singapore Swimming Club,Account No. with billing corporation as shown ...,5 Characters (2 Alphabet 3 Digits),TA123,Country Clubs / Recreational Clubs
4,Singapore Island Country Club,Club No. as provided by billing organisation,5 Characters (1 Alphabet 4 Digits),A0123400 as A1234,Country Clubs / Recreational Clubs
...,...,...,...,...,...
99,StarHub Ltd,Account No. with StarHub Ltd as displayed on y...,"Numeric with decimals, ending with a letter.",Eg: 1.12345678A,Telecommunications and Utilities
100,Sembcorp Power Pte Ltd,Account No. with Sembcorp Power Pte Ltd as dis...,10 Characters (1 Uppercase alphabet & 9 digits),H123456782 / B000000018,Telecommunications and Utilities
101,Sunseap Energy Pte Ltd,Sunseap Account number as shown on top right o...,Sunseap Account number as shown on top right o...,23012345,Telecommunications and Utilities
102,Union Power Pte Ltd,MSSL Number,10 Digits / 13 Digits,9308014458 9308014458-013,Telecommunications and Utilities
